In [40]:
import pandas_datareader as pdr
import datetime
import pandas as pd
import yfinance as yf

## Question 1

In [21]:
start_date = datetime.date(2023 - 1, 1, 1)
gdp_data = pdr.DataReader("GDPC1", "fred", start=start_date)
gdp_data

,GDPC1
DATE,
2022-01-01,21738.871
2022-04-01,21708.160
2022-07-01,21851.134
2022-10-01,21989.981
2023-01-01,22112.329
2023-04-01,22225.350
2023-07-01,22490.692
2023-10-01,22679.255


In [25]:
gdp_data["GDPC1_yoy"] = (gdp_data["GDPC1"] / gdp_data["GDPC1"].shift(4)) - 1.0

growth_rate_2023 = gdp_data.loc[gdp_data.index.year == 2023, "GDPC1_yoy"]
growth_rate_2023

DATE
2023-01-01    0.017179
2023-04-01    0.023825
2023-07-01    0.029269
2023-10-01    0.031345
Name: GDPC1_yoy, dtype: float64

In [23]:
growth_rate_2023.mean() * 100

2.5404433928676253

## Question 2

In [34]:
start_date = datetime.date(2000, 1, 1)
dgs2 = pdr.DataReader("DGS2", "fred", start=start_date)
dgs10 = pdr.DataReader("DGS10", "fred", start=start_date)
dgs_2_10 = dgs2.join(dgs10)
dgs_2_10

,DGS2,DGS10
DATE,,
2000-01-03,6.38,6.58
2000-01-04,6.30,6.49
2000-01-05,6.38,6.62
2000-01-06,6.35,6.57
2000-01-07,6.31,6.52
...,...,...
2024-04-05,4.73,4.39
2024-04-08,4.78,4.42
2024-04-09,4.74,4.36


In [39]:
(dgs_2_10["DGS10"] - dgs_2_10["DGS2"]).min()

-1.0800000000000005

In [55]:
sp500 = yf.download(tickers = "^GSPC", period="max", interval="1d")
mxx = yf.download(tickers = "^MXX", period="max", interval="1d")
sp500

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1927-12-30,17.660000,17.660000,17.660000,17.660000,17.660000,0
1928-01-03,17.760000,17.760000,17.760000,17.760000,17.760000,0
1928-01-04,17.719999,17.719999,17.719999,17.719999,17.719999,0
1928-01-05,17.549999,17.549999,17.549999,17.549999,17.549999,0
1928-01-06,17.660000,17.660000,17.660000,17.660000,17.660000,0
...,...,...,...,...,...,...
2024-04-09,5217.029785,5224.810059,5160.779785,5209.910156,5209.910156,3400680000
2024-04-10,5167.879883,5178.430176,5138.700195,5160.640137,5160.640137,3845930000
2024-04-11,5172.950195,5211.779785,5138.770020,5199.060059,5199.060059,3509380000


In [60]:
end_date = "2024-04-09"
start_date = "2019-04-09"
five_y_growth = lambda ticker: (ticker.loc[end_date, "Close"] / ticker.loc[start_date, "Close"]) - 1

max(five_y_growth(sp500), five_y_growth(mxx)) * 100

81.01279426847174

In [62]:
# In the exercise gave 6 tickers, JPM shouldn't be there.
top5_tickers = ["2222.SR","BRK-B", "AAPL", "MSFT", "GOOG", "JPM"]
range_ratio = [0.0] * 6
for i, ticker in enumerate(top5_tickers):
    adj_close_history = yf.download(tickers = ticker, period="max", interval="1d", 
                                    start="2023-01-01", end="2024-01-01")["Adj Close"]
    range_ratio[i] = 1.0 - (adj_close_history.min() / adj_close_history.max())

max(range_ratio)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


0.4242066515530232

In [68]:
yields_2023 = [0.0] * 6

def dividend_sum(ticker):
    ticker_class = yf.Ticker(ticker)
    dividend = ticker_class.dividends
    return dividend.loc[dividend.index.year == 2023].sum()

def last_price(ticker):
    stock_data = yf.download(ticker, start="2023-01-01", end="2023-12-31")
    return stock_data["Adj Close"].iat[-1]

for i, ticker in enumerate(top5_tickers):
    yields_2023[i] = dividend_sum(ticker) / last_price(ticker)

max(yields_2023) * 100

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


2.770149819503256